In [ ]:
import pymc3 as pm
import math
import numpy as np
import theano
import arviz as az
import pandas as pd
from tqdm.auto import tqdm
import pickle
import matplotlib.pyplot as plt

np.random.seed(42)
pm.set_tt_rng(42)

In [ ]:
away_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/all_games_away.csv")
print(away_data.head())
print(away_data.shape)

In [ ]:
traces_dicts = []
traces = []
(
    last_mu1,
    last_mu2,
    last_mu3,
    last_mu4,
    last_mu5,
    last_mu6,
    last_mu7,
    last_mu8,
    last_t,
) = (0, 0, 0, 0, 0, 0, 0, 0, 0)

for ii in tqdm(range(100)):
    R_away = np.empty((950))
    X_away = np.empty((9, 950))

    ## create X
    for i, row in away_data.iterrows():
        if int(i / 100) < 950:
            for x in range(len(row)):
                if int(row[0]) == ii:
                    if x == 9:
                        R_away[int(i / 100)] = int(row[9])
                        continue
                    X_away[x][int(i / 100)] = row[x]

    with pm.Model() as away_model:
        at1 = pm.Normal("at1", mu=last_mu1, sigma=math.sqrt(2))
        at2 = pm.Normal("at2", mu=last_mu2, sigma=math.sqrt(2))
        at3 = pm.Normal("at3", mu=last_mu3, sigma=math.sqrt(2))
        at4 = pm.Normal("at4", mu=last_mu4, sigma=math.sqrt(2))
        at5 = pm.Normal("at5", mu=last_mu5, sigma=math.sqrt(2))
        at6 = pm.Normal("at6", mu=last_mu6, sigma=math.sqrt(2))
        at7 = pm.Normal("at7", mu=last_mu7, sigma=math.sqrt(2))
        at8 = pm.Normal("at8", mu=last_mu8, sigma=math.sqrt(2))
        # at9 = pm.Normal("at9", mu=last_mu9, sigma=math.sqrt(2))
        # at = pm.GaussianRandomWalk("at", mu=0, sigma=math.sqrt(2), shape=(100,9))
        beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

        data = pm.Data("data", X_away)
        # theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
        theta = pm.invlogit(
            (
                at1 * data[1]
                + at2 * data[2]
                + at3 * data[3]
                + at4 * data[4]
                + at5 * data[5]
                + at6 * data[6]
                + at7 * data[7]
                + at8 * data[8]
            )
            + beta
        )
        # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)
        # t = pm.Normal("t", mu=last_t, sigma=math.sqrt(2))
        like = pm.Poisson("like", theta, observed=R_away)

    # pm.model_to_graphviz(away_model)

    with away_model:
        approx = pm.fit(
            100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)]
        )
        idata_advi = az.from_pymc3(approx.sample(500))

    trace = idata_advi.to_dict()
    last_mu1 = np.mean(trace["posterior"]["at1"][0])
    last_mu2 = np.mean(trace["posterior"]["at2"][0])
    last_mu3 = np.mean(trace["posterior"]["at3"][0])
    last_mu4 = np.mean(trace["posterior"]["at4"][0])
    last_mu5 = np.mean(trace["posterior"]["at5"][0])
    last_mu6 = np.mean(trace["posterior"]["at6"][0])
    last_mu7 = np.mean(trace["posterior"]["at7"][0])
    last_mu8 = np.mean(trace["posterior"]["at8"][0])
    # last_t = np.mean(trace['posterior']['t'][0])
    traces_dicts.append(trace)
    traces.append(idata_advi)
    print(ii)

In [ ]:
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/away_traces_100_models_verbesserung_ohne_t.pkl",
    "wb",
) as f:
    pickle.dump(traces, f)
f.close()
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/away_traces_dicts_100_models_verbesserung_ohne_t.pkl",
    "wb",
) as f:
    pickle.dump(traces_dicts, f)
f.close()

In [ ]:
home_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/all_games_home.csv")
print(home_data.head())
print(home_data.shape)

In [ ]:
traces_dicts = []
traces = []
(
    last_mu1,
    last_mu2,
    last_mu3,
    last_mu4,
    last_mu5,
    last_mu6,
    last_mu7,
    last_mu8,
    last_t,
) = (0, 0, 0, 0, 0, 0, 0, 0, 0)

for ii in tqdm(range(100)):
    R_home = np.empty((950))
    X_home = np.empty((9, 950))

    ## create X
    for i, row in home_data.iterrows():
        if int(i / 100) < 950:
            for x in range(len(row)):
                if int(row[0]) == ii:
                    if x == 9:
                        R_home[int(i / 100)] = int(row[9])
                        continue
                    X_home[x][int(i / 100)] = row[x]

    with pm.Model() as home_model:
        at1 = pm.Normal("at1", mu=last_mu1, sigma=math.sqrt(2))
        at2 = pm.Normal("at2", mu=last_mu2, sigma=math.sqrt(2))
        at3 = pm.Normal("at3", mu=last_mu3, sigma=math.sqrt(2))
        at4 = pm.Normal("at4", mu=last_mu4, sigma=math.sqrt(2))
        at5 = pm.Normal("at5", mu=last_mu5, sigma=math.sqrt(2))
        at6 = pm.Normal("at6", mu=last_mu6, sigma=math.sqrt(2))
        at7 = pm.Normal("at7", mu=last_mu7, sigma=math.sqrt(2))
        at8 = pm.Normal("at8", mu=last_mu8, sigma=math.sqrt(2))
        # at9 = pm.Normal("at9", mu=last_mu9, sigma=math.sqrt(2))
        # at = pm.GaussianRandomWalk("at", mu=0, sigma=math.sqrt(2), shape=(100,9))
        ha = pm.Normal("ha", mu=0, sigma=math.sqrt(2))
        beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

        data = pm.Data("data", X_home)
        # theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
        theta = pm.invlogit(
            (
                at1 * data[1]
                + at2 * data[2]
                + at3 * data[3]
                + at4 * data[4]
                + at5 * data[5]
                + at6 * data[6]
                + at7 * data[7]
                + at8 * data[8]
            )
            + beta
            + ha
        )
        # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)
        # t = pm.Normal("t", mu=last_t, sigma=math.sqrt(2))
        like = pm.Poisson("like", theta, observed=R_home)

    # pm.model_to_graphviz(away_model)

    with home_model:
        approx = pm.fit(
            100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)]
        )
        idata_advi = az.from_pymc3(approx.sample(500))

    trace = idata_advi.to_dict()
    last_mu1 = np.mean(trace["posterior"]["at1"][0])
    last_mu2 = np.mean(trace["posterior"]["at2"][0])
    last_mu3 = np.mean(trace["posterior"]["at3"][0])
    last_mu4 = np.mean(trace["posterior"]["at4"][0])
    last_mu5 = np.mean(trace["posterior"]["at5"][0])
    last_mu6 = np.mean(trace["posterior"]["at6"][0])
    last_mu7 = np.mean(trace["posterior"]["at7"][0])
    last_mu8 = np.mean(trace["posterior"]["at8"][0])
    # last_t = np.mean(trace['posterior']['t'][0])
    traces_dicts.append(trace)
    traces.append(idata_advi)
    print(ii)

In [ ]:
import pickle

with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/home_traces_100_models_verbesserung_ohne_t.pkl",
    "wb",
) as f:
    pickle.dump(traces, f)
f.close()
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/home_traces_dicts_100_models_verbesserung_ohne_t.pkl",
    "wb",
) as f:
    pickle.dump(traces_dicts, f)
f.close()

In [ ]:
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/away_traces_100_models_verbesserung_ohne_t.pkl",
    "rb",
) as f:
    away_traces = pickle.load(f)
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/away_traces_dicts_100_models_verbesserung_ohne_t.pkl",
    "rb",
) as f:
    away_dicts = pickle.load(f)
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/home_traces_100_models_verbesserung_ohne_t.pkl",
    "rb",
) as f:
    home_traces = pickle.load(f)
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/home_traces_dicts_100_models_verbesserung_ohne_t.pkl",
    "rb",
) as f:
    home_dict = pickle.load(f)

In [ ]:
name = "t"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at1"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at2"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at3"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at4"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at5"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at6"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at7"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "at8"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")

In [ ]:
name = "beta"
tmp = []
fig = plt.figure()
ax = fig.subplots()
for i in range(100):
    vals = away_dicts[i]["posterior"][name]
    ax.vlines([i], np.min(vals), np.max(vals), color="grey")
    tmp.append(np.mean(vals))
ax.plot(tmp, color="black")